# Data Preparation for Power BI Report

This notebook cleans and aggregates the data for ingesting into the power BI report.

Otherwise the data is too big and noisy

In [5]:
import time
import importlib
import src.imdb_views as imdb_views
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

_ = importlib.reload(imdb_views)

In [2]:
reader = imdb_views.IMDBDataReader("../../PublicDatasets/IMDB/")

In [3]:
ratings = reader.get_ratings()
title_basics = reader.get_title_basics()

# take intersection of titles with ratings and basic information to reduce the size of the dataset
titles_with_ratings_and_basic_info = \
    ratings[['tconst']].drop_duplicates()\
    .merge(title_basics[['tconst']].drop_duplicates(), on='tconst', how='inner')
ratings = ratings.merge(titles_with_ratings_and_basic_info, on='tconst', how='inner')
title_basics = title_basics.merge(titles_with_ratings_and_basic_info, on='tconst', how='inner')

title_regions = reader.get_title_regions(titles_to_fetch=titles_with_ratings_and_basic_info)

# keep only titles with regions
titles_w_ratings_basic_info_and_regions = \
    titles_with_ratings_and_basic_info\
    .merge(title_regions[['tconst']].drop_duplicates(), on='tconst', how='inner')

ratings = ratings.merge(titles_w_ratings_basic_info_and_regions, on='tconst', how='inner')
title_basics = title_basics.merge(titles_w_ratings_basic_info_and_regions, on='tconst', how='inner')
title_regions = title_regions.merge(titles_w_ratings_basic_info_and_regions, on='tconst', how='inner')

title_to_names = reader.get_title_to_names(titles_to_fetch=titles_w_ratings_basic_info_and_regions)

Time taken to read ratings data: 0.625375509262085 seconds
Time taken to read title basics data: 18.166292428970337 seconds
Time taken to read title regions data: 68.9937744140625 seconds
Time taken to compute the final title to names dataframe: 1.965601623058319 mins


In [6]:
title_to_region_of_origin = pd.read_csv('data/region_of_origin_per_title.tsv', sep='\t')

In [8]:
# get a list of genres per titleType

genre_list = title_basics[['titleType', 'genres']].drop_duplicates()
genre_list['genres'] = genre_list['genres'].str.split(',')
genre_list = genre_list.explode('genres')\
    .drop_duplicates()\
    .sort_values(by=['titleType', 'genres'])\
    .query('genres != "\\\\N"')\
    .query('genres != "Adult"')
genre_list.head()

,titleType,genres
2,movie,Action
2,movie,Adventure
595,movie,Animation
2,movie,Biography
8,movie,Comedy


### Top Actors / Movies Data Prep

This section preps the data for the power bi application to show the top actors and movies.

Power bi components to support:

1. [Genre|All] [TitleType] [Region] [Language] [RegionOfOrigin] Top movies/shows [Slider=year] (by Rating / NumVotes)
2. [Genre|All] [TitleType] [Region] [Language] [RegionOfOrigin] Top artists [Slider=year] (by Rating / NumVotes)
3. [Genre|All] [TitleType] [Region] [Language] [RegionOfOrigin] Top artist-artist pairings [Slider=year]

In [9]:
# [Genre|All] [TitleType] [Region] [Language] Top-N movies/shows (bar plot with the poster on top) [Slider=year]
N = 20
top_movies_df = []

for title_type in ['movie', 'tvSeries']:
    titles_of_type = title_basics\
        .query(f'titleType == "{title_type}"')\
        .copy()\
        .query('startYear != "\\\\N"')\
        .astype({'startYear': 'int'})\
        .merge(ratings, on='tconst')\
        .merge(title_regions, on='tconst')\
        .merge(title_to_region_of_origin, on='tconst')

    for genre in ['ALL'] + genre_list.query(f'titleType == "{title_type}"')['genres'].tolist():
        titles_of_type_genre = titles_of_type
        if genre!='ALL':
            titles_of_type_genre = titles_of_type[titles_of_type['genres'].apply(lambda x: genre in x.split(','))]

        for metric in ['numVotes', 'averageRating']:
            top_titles = titles_of_type_genre.copy()
            top_titles['rank1'] = top_titles\
                .groupby(['startYear', 'region', 'language'])['numVotes']\
                .rank(ascending=False, method='first')
            top_titles['rank2'] = top_titles\
                .groupby(['startYear', 'RegionOfOrigin', 'language'])['numVotes']\
                .rank(ascending=False, method='first')
            if metric == 'averageRating':
                top_titles = top_titles.query('rank1 < 1000 and numVotes > 1000')
                top_titles['rank1'] = top_titles\
                    .groupby(['startYear', 'region', 'language'])['averageRating']\
                    .rank(ascending=False, method='first')
                top_titles['rank2'] = top_titles\
                    .groupby(['startYear', 'RegionOfOrigin', 'language'])['averageRating']\
                    .rank(ascending=False, method='first')

            top_titles = top_titles.query('rank1 <= @N or rank2 <= @N')
            top_titles['PopularityMetric'] = metric
            top_titles['Popularity'] = top_titles[metric].astype('float')
            top_titles['Genre'] = genre
            top_titles['Type'] = title_type
            top_movies_df.append(top_titles)
        print (f"Processed {title_type} - {genre}")

top_movies_df = pd.concat(top_movies_df)

Processed movie - ALL
Processed movie - Action
Processed movie - Adventure
Processed movie - Animation
Processed movie - Biography
Processed movie - Comedy
Processed movie - Crime
Processed movie - Documentary
Processed movie - Drama
Processed movie - Family
Processed movie - Fantasy
Processed movie - Film-Noir
Processed movie - Game-Show
Processed movie - History
Processed movie - Horror
Processed movie - Music
Processed movie - Musical
Processed movie - Mystery
Processed movie - News
Processed movie - Reality-TV
Processed movie - Romance
Processed movie - Sci-Fi
Processed movie - Sport
Processed movie - Talk-Show
Processed movie - Thriller
Processed movie - War
Processed movie - Western
Processed tvSeries - ALL
Processed tvSeries - Action
Processed tvSeries - Adventure
Processed tvSeries - Animation
Processed tvSeries - Biography
Processed tvSeries - Comedy
Processed tvSeries - Crime
Processed tvSeries - Documentary
Processed tvSeries - Drama
Processed tvSeries - Family
Processed tvS

In [10]:
top_movies_df.to_csv('data/top_movies.tsv', sep='\t', index=False)

In [26]:
# [Genre|All] [TitleType] [Region] [Language] [AreTopUSTitlesExcluded] Top-N actors per year (word cloud) [Slider=year] (by Rating / NumVotes)
N = 20
top_artists_df = []

release_regions = title_regions.copy()
release_regions['RegionType'] = 'RegionOfRelease'
origin_regions = title_to_region_of_origin.copy()\
    .rename(columns={'RegionOfOrigin': 'region'})
origin_regions['language'] = "ALL"
origin_regions['RegionType'] = 'RegionOfOrigin'
regions = pd.concat([release_regions, origin_regions])

for title_type in ['movie', 'tvSeries']:
    start_time1 = time.time()
    titles_of_type = title_basics\
        .query(f'titleType == "{title_type}"')\
        .query('startYear != "\\\\N"')\
        .astype({'startYear': 'int'})\
        .merge(ratings, on='tconst')\
        .merge(regions, on='tconst')

    for genre in ['ALL'] + genre_list.query(f'titleType == "{title_type}"')['genres'].tolist():
        start_time2 = time.time()
        titles_of_type_genre = titles_of_type
        if genre!='ALL':
            titles_of_type_genre = titles_of_type[titles_of_type['genres']\
                                                    .apply(lambda x: genre in x.split(','))]

        for category in ['ALL', 'actor', 'actress', 'director']:
            category_artists = title_to_names.copy()
            if category!='ALL':
                category_artists = category_artists.query(f'category == "{category}"')
            else:
                category_artists['category'] = 'ALL'

            top_artists = titles_of_type_genre\
                .merge(category_artists, on='tconst')\
                .groupby(['category', 'startYear', 'RegionType', 'region', 'language', 'primaryName'])\
                .agg({'numVotes': 'sum'})\
                .reset_index()
            top_artists['rank'] = top_artists\
                .groupby(['category', 'startYear', 'RegionType', 'region', 'language'])['numVotes']\
                .rank(ascending=False, method='first')
            top_artists = top_artists.query('rank <= @N')\
                [['category', 'startYear', 'RegionType', 'region', 'language', 'primaryName']]

            top_artists_titles = titles_of_type_genre\
                .merge(category_artists, on='tconst')\
                .merge(top_artists, on=['category', 'startYear', 'RegionType', 'region', 'language', 
                        'primaryName'])

            # keep only max top-3 titles of the artist per year. Club rest into "Others"
            top_artists_titles['TitleRankPerArtist'] = top_artists_titles\
                .groupby(['category', 'startYear', 'RegionType', 'region', 'language', 'primaryName'])['numVotes']\
                .rank(ascending=False, method='first')

            top_artists_titles['primaryTitle'] = top_artists_titles\
                .apply(lambda r: r['primaryTitle'] if r['TitleRankPerArtist']<=3 else "Others", axis=1)
            top_artists = top_artists_titles\
                .groupby(['category', 'startYear', 'RegionType', 'region', 'language', 
                            'primaryName', 'primaryTitle'])\
                .agg({'numVotes': 'sum', 'tconst': 'min'})\
                .reset_index()

            top_artists = top_artists\
                .rename(columns={'primaryName': 'ArtistName', 'category': 'ArtistType'})
            top_artists['Genre'] = genre
            top_artists['ArtistType'] = category
            top_artists['Type'] = title_type
            top_artists_df.append(top_artists)
        print (f'Time taken for {title_type} and {genre}: {(time.time()-start_time2) / 60} mins')
    print (f'Time taken for {title_type}: {(time.time()-start_time1) / 60} mins')

top_artists_df = pd.concat(top_artists_df)

Time taken for movie and ALL: 1.7911941528320312 mins
Time taken for movie and Action: 0.49212480783462526 mins
Time taken for movie and Adventure: 0.40286142826080323 mins
Time taken for movie and Animation: 0.2374368627866109 mins
Time taken for movie and Biography: 0.2676830728848775 mins
Time taken for movie and Comedy: 0.7014366745948791 mins
Time taken for movie and Crime: 0.4675933480262756 mins
Time taken for movie and Documentary: 0.23865835666656493 mins
Time taken for movie and Drama: 1.115192755063375 mins
Time taken for movie and Family: 0.31214715242385865 mins
Time taken for movie and Fantasy: 0.30899893840154014 mins
Time taken for movie and Film-Noir: 0.13614017566045125 mins
Time taken for movie and Game-Show: 0.08883924086888631 mins
Time taken for movie and History: 0.27248462041219074 mins
Time taken for movie and Horror: 0.401133660475413 mins
Time taken for movie and Music: 0.2635766347249349 mins
Time taken for movie and Musical: 0.2683478593826294 mins
Time tak

In [27]:
top_artists_df.to_csv('data/top_artists.tsv', sep='\t', index=False)

# Genre Analysis Data Preparation

This section preps the data for the genre analysis powerbi report.
Report includes these analysis:

Filters = [title type] [year range] [region type] [region] [language] [traffic type]

1. Genre share bar chart
    - traffic type is either: number of titles, number of votes
    - also show line chart per year for the selected range
2. Genre-Genre heatmap 
    - H[i, j] = (traffic common between genre i and j) / (traffic in genre i)
3. Box plot of duration for different genres 

In [66]:
def try_convert_int(n, default=0):
    try:
        return int(n)
    except:
        return default

def get_genres(genres_str):
    genres_str = genres_str.replace('\\N', '')
    if genres_str == '': return ["ALL"]
    return genres_str.split(',') + ["ALL"]

genre_stats_df = []

release_regions = title_regions.copy()
release_regions['RegionType'] = 'RegionOfRelease'
origin_regions = title_to_region_of_origin.copy()\
    .rename(columns={'RegionOfOrigin': 'region'})
origin_regions['language'] = "ALL"
origin_regions['RegionType'] = 'RegionOfOrigin'
regions = pd.concat([release_regions, origin_regions])

for title_type in ['movie', 'tvSeries']:
    start_time1 = time.time()
    genre_stats = title_basics\
        .query(f'titleType == "{title_type}"')\
        .query('startYear != "\\\\N"')\
        .astype({'startYear': 'int'})\
        .merge(ratings, on='tconst')\
        .merge(regions, on='tconst')

    genre_stats['runtime'] = genre_stats['runtimeMinutes'].apply(try_convert_int)
    genre_stats['runtimeSquare'] = genre_stats['runtime'] * genre_stats['runtime']

    genre_stats1 = genre_stats\
        .groupby(['startYear', 'RegionType', 'region', 'language', 'genres'])\
        .agg({'numVotes': 'sum', 'tconst': 'count'})\
        .reset_index()\
        .rename(columns={'tconst': 'numTitles'})
    
    # compute stats needed to compute mean and variance later on
    genre_stats2 = genre_stats\
        .query('runtime != 0')\
        .groupby(['startYear', 'RegionType', 'region', 'language', 'genres'])\
        .agg({'tconst': 'count', 'runtime': 'sum', 'runtimeSquare': 'sum'})\
        .reset_index()\
        .rename(columns={'tconst': 'runtimeCount', 'runtime': 'runtimeSum', 'runtimeSquare': 'runtimeSquareSum'})
    
    genre_stats = genre_stats1.merge(genre_stats2, on=['startYear', 'RegionType', 'region', 'language', 'genres'])
    genre_stats['genre'] = genre_stats['genres'].apply(get_genres)
    genre_stats = genre_stats\
        .explode('genre')\
        .groupby(['startYear', 'RegionType', 'region', 'language', 'genre'])\
        .agg({'numVotes': 'sum', 'numTitles': 'sum', 'runtimeCount': 'sum', 'runtimeSum': 'sum', 'runtimeSquareSum': 'sum'})\
        .reset_index()
    genre_stats['Type'] = title_type
    
    genre_stats_df.append(genre_stats)
    print (f'Time taken for {title_type}: {(time.time()-start_time1) / 60} mins')

genre_stats_df = pd.concat(genre_stats_df)


Time taken for movie: 0.07811247110366822 mins
Time taken for tvSeries: 0.025231122970581055 mins


In [67]:
genre_stats_df.to_csv("data/genre_stats.tsv", sep='\t', index=False)

In [74]:
def try_convert_int(n, default=0):
    try:
        return int(n)
    except:
        return default

def get_genre_pairs(genres_str):
    genres_str = genres_str.replace('\\N', '')
    if genres_str == '': return None
    genres = genres_str.split(',')
    return [','.join([g1, g2]) for g1 in genres for g2 in genres]

genre_pair_stats_df = []

release_regions = title_regions.copy()
release_regions['RegionType'] = 'RegionOfRelease'
origin_regions = title_to_region_of_origin.copy()\
    .rename(columns={'RegionOfOrigin': 'region'})
origin_regions['language'] = "ALL"
origin_regions['RegionType'] = 'RegionOfOrigin'
regions = pd.concat([release_regions, origin_regions])

for title_type in ['movie', 'tvSeries']:
    start_time1 = time.time()
    genre_stats = title_basics\
        .query(f'titleType == "{title_type}"')\
        .query('startYear != "\\\\N"')\
        .astype({'startYear': 'int'})\
        .merge(ratings, on='tconst')\
        .merge(regions, on='tconst')

    genre_stats['runtime'] = genre_stats['runtimeMinutes'].apply(try_convert_int)
    genre_stats['runtimeSquare'] = genre_stats['runtime'] * genre_stats['runtime']

    genre_pair_stats = genre_stats\
        .groupby(['startYear', 'RegionType', 'region', 'language', 'genres'])\
        .agg({'numVotes': 'sum', 'tconst': 'count'})\
        .reset_index()\
        .rename(columns={'tconst': 'numTitles'})
    
    genre_pair_stats['genre_pair'] = genre_pair_stats['genres'].apply(get_genre_pairs)
    genre_pair_stats = genre_pair_stats\
        .explode('genre_pair')\
        .dropna(subset=['genre_pair'])\
        .groupby(['startYear', 'RegionType', 'region', 'language', 'genre_pair'])\
        .agg({'numVotes': 'sum', 'numTitles': 'sum'})\
        .reset_index()
    
    unique_genre_pairs = genre_pair_stats[['genre_pair']].drop_duplicates()
    unique_filter_values = genre_pair_stats[['startYear', 'RegionType', 'region', 'language']].drop_duplicates()
    unique_genre_pairs['Key'] = 1
    unique_filter_values['Key'] = 1
    unique_combos = unique_genre_pairs.merge(unique_filter_values, on='Key').drop(columns='Key')
    genre_pair_stats = unique_combos\
        .merge(genre_pair_stats, on=['startYear', 'RegionType', 'region', 'language', 'genre_pair'], how='left')\
        .fillna(0)

    genre_pair_stats['genre1'] = genre_pair_stats['genre_pair'].apply(lambda x: x.split(',')[0])
    genre_pair_stats['genre2'] = genre_pair_stats['genre_pair'].apply(lambda x: x.split(',')[1])
    
    genre_pair_stats['Type'] = title_type

    genre_pair_stats_df.append(genre_pair_stats)
    print (f'Time taken for {title_type}: {(time.time()-start_time1) / 60} mins')

genre_pair_stats_df = pd.concat(genre_pair_stats_df)


# add total votes per slice in genre1 as a separate column
genre1_votes = genre_stats_df[['Type', 'startYear', 'RegionType', 'region', 'language', 'genre', 'numVotes']]\
    .rename(columns={'numVotes': 'totalGenre1Votes', 'genre': 'genre1'})

genre_pair_stats_df = genre_pair_stats_df\
    .merge(genre1_votes, on=['Type', 'startYear', 'RegionType', 'region', 'language', 'genre1'])

Time taken for movie: 0.1236090858777364 mins
Time taken for tvSeries: 0.06383922100067138 mins


In [75]:
genre_pair_stats_df.to_csv("data/genre_pair_stats.tsv", sep='\t', index=False)